In [1]:
import numpy as np
import math
from enum import Enum
from collections import namedtuple

# matplotlib
# ライブラリーのインポート
import matplotlib.pyplot as plt
# インライン表示
%matplotlib inline

# 日本語フォント
import pylab
from matplotlib.font_manager import FontProperties
# 日本語フォント
fp = FontProperties(fname='C:\WINDOWS\Fonts\msgothic.ttc', size = 12)

# 第九章　自然エネルギー利用設備

## 第三節　空気集熱式太陽熱利用設備

### 5. 給湯部における補正集熱量

日付$d$における1日当たりの給湯部における補正集熱量$L_{sun,ass,d}$

$$
\begin{equation*}
    L_{sun,acc,d} = min \left( L_{tunk,d} , \: \left( L'_{k,d} + L'_{s,d} + L'_{w,d} + L'_{b1,d} + L'_{b2,d} + L'_{ba1,d} \right) \times f_{sr,uplim} \right)
\end{equation*}
$$

$f_{sr,uplim}$ : 給湯部の分担率上限値(-)   
$L_{sun,acc,d}$ : 日付$d$における1日当たりの給湯部における補正集熱量(MJ/日)  
$L_{tnk,d}$ : 日付$d$における1日当たりの給湯部におけるタンク蓄熱量の上限による補正集熱量(MJ/日)  
$L'_{k,d}$ : 日付$d$における1日当たりの台所水栓における節湯補正給湯熱負荷(MJ/日)  
$L'_{s,d}$ : 日付$d$における1日当たりの浴室シャワー水栓における節湯補正給湯熱負荷(MJ/日)  
$L'_{w,d}$ : 日付$d$における1日当たりの洗面水栓における節湯補正給湯熱負荷(MJ/日)  
$L'_{b1,d}$ : 日付$d$における1日当たりの浴槽水栓湯はり時における節湯補正給湯熱負荷(MJ/日)  
$L'_{b2,d}$ : 日付$d$における1日当たりの浴槽自動湯はり時における節湯補正給湯熱負荷(MJ/日)  
$L'_{ba1,d}$ : 日付$d$における1日当たりの浴槽水栓さし湯時における節湯補正給湯熱負荷(MJ/日)

給湯部の分担率上限値$f_{sr,uplim}$

$$
\begin{equation*}
    f_{sr,uplim} = 0.9
\end{equation*}
$$

日付$d$における1日当たりの給湯部におけるタンク蓄熱量の上限による補正集熱量$L_{tnk,d}$

$$
\begin{equation*}
   L_{tnk,d} = min \left( Q_{d} , \: HC_{tnk,d} \times \alpha_{tnk,d} \right)
\end{equation*}
$$

$L_{tnk,d}$ : 日付$d$における1日当たりの給湯部におけるタンク蓄熱量の上限による補正集熱量(MJ/日)  
$Q_{d}$ : 日付$d$における1日当たりの基準集熱量(MJ/日)  
$HC_{tnk,d}$ : 日付$d$における給湯部のタンク蓄熱量の上限(MJ)  
$\alpha_{tnk,d}$ : 日付$d$における給湯部のタンク有効利用率(1/日)  

日付$d$における給湯部のタンク有効利用率$\alpha_{tnk,d}$

$$
\begin{equation*}
    \alpha_{tnk,d} = 1.0
\end{equation*}
$$

日付$d$における給湯部のタンク蓄熱量の上限$HC_{tnk,d}$

$$
\begin{equation*}
    HC_{tnk,d} = \left( \theta_{tnk} - \theta_{wtr,d} \right) \times W_{tnk} \times 10^{-3}
\end{equation*}
$$

$HC_{tnk,d}$ : 日付$d$における給湯部のタンク蓄熱量の上限(MJ)  
$\theta_{tnk}$ : 給湯部のタンク内温度(℃)   
$\theta_{wtr,d}$ : 日付$d$における日平均給水温度(℃)  
$W_{tnk}$ : 給湯部のタンク容量(L)  

給湯部のタンク内温度$\theta_{tnk}$

$$
\begin{equation*}
    \theta_{tnk} = 65
\end{equation*}
$$

### 6. 基準集熱量

日付$d$における1日当たりの基準集熱量$Q_{d}$

$$
\begin{equation*}
    Q_{d} = \sum_{t=0}^{23} \left( Q_{col,d,t} \times f_{hx} \times f_{s} \times t_{cp,d,t} \right)
\end{equation*}
$$

$f_{hx}$ : 給湯部の熱交換効率(-)  
$f_{s}$ : 給湯部のシステム効率(-)  
$Q_{d}$ : 日付$d$における1日当たりの基準集熱量(MJ/日)  
$Q_{col,d,t}$ : 日付$d$の時刻$t$における1時間当たりの集熱部における集熱量(MJ/h)   
$t_{cp,d,t}$ : 日付$d$の時刻$t$における1時間当たりの循環ポンプの稼働時間(h/h)   

給湯部の熱交換効率$f_{hx}$

$$
\begin{equation*}
    f_{hx} = 0.25
\end{equation*}
$$

給湯部のシステム効率$f_{s}$

$$
\begin{equation*}
    f_{s} = 0.85
\end{equation*}
$$